In [10]:
%reload_ext autoreload
%autoreload 2

In [11]:
import json
file = open('./../r_addresses.json')
wallet = json.load(file)

In [12]:
# fund transaction
from key import ECPubKey,ECKey
from messages import CTxIn,COutPoint,CTxOut,CTransaction,CTxWitness,CTxInWitness,ser_uint256
from script import tagged_hash, sha256,taproot_construct,SegwitV0SignatureHash, CScript, OP_1,OP_2, OP_CHECKMULTISIG, OP_0,OP_CHECKSIG
import sys
sys.path.append("..")
from ecc import PrivateKey

prev_tx_hx = int("7c4e6957c0b39023414d60ec21a4a4194347d6d6f2353c6b1cabbcc284b71ef3", 16)
prv_amount=int(100*100000000)
prv_pub_key_hash = "7d7143a28a3b836e3043f3ba5dbcdc4ee29b7457"
prv_script=bytes.fromhex(f"76a914{prv_pub_key_hash}88ac")
prev_index = 1

p=ECKey()
p.set(0x00db5cdc1663eea25406126e642f72ef7c78f68d249589d2873c613e61929eea)

target_public_key_bytes1=bytes.fromhex(wallet[12]["public key"])
target_public_key_bytes2=bytes.fromhex(wallet[11]["public key"])
target_amount = int(99.999*100000000)

tx_in =CTxIn(COutPoint(prev_tx_hx, prev_index))

target_public_key1=ECPubKey().set(target_public_key_bytes1)
target_public_key2=ECPubKey().set(target_public_key_bytes2)
mast_scripts = [ CScript([OP_2,target_public_key1.get_bytes(),target_public_key2.get_bytes(),OP_2, OP_CHECKMULTISIG]) ]

locking_script, tweak, control_map = taproot_construct(target_public_key1, mast_scripts)

tx_outs = CTxOut(target_amount, locking_script)

tx = CTransaction()
tx.vin=[tx_in]
tx.vout=[tx_outs]
tx.calc_sha256()
tx_sig_hash = SegwitV0SignatureHash(prv_script, tx, 0, 0x01, prv_amount)
tx_input_0_wit = CTxInWitness([p.sign_ecdsa(tx_sig_hash) + (1).to_bytes(1, 'big'), p.get_pubkey().get_bytes(False)])
tx.wit.vtxinwit = [tx_input_0_wit]

print(tx.serialize_with_witness().hex())

# Use for Next Transaction
tx1_outs = tx.vout
t = tweak
cmp = control_map


01000000000101f31eb784c2bcab1c6b3c35f2d6d6474319a4a421ec604d412390b3c057694e7c0100000000ffffffff01605d0a5402000000225120a873329f62867d05459576010d8e5bcbe5198c05007a09d75146f7890e62e49a0247304402201ebd6d0a828e68da94f5621b2329e3f69c2672cbfcfed646f0d34d3c7fc4177602200372556b06b206d067c14bc72d7943243f7d6633ed6ac04e2d34c4dabb75bc0e012103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee00000000


In [14]:
# lock p2tr script
from key import ECPubKey,ECKey
from messages import CTxIn,COutPoint,CTxOut,CTransaction,CTxWitness,CTxInWitness,ser_uint256
from script import tagged_hash, sha256,taproot_construct,SegwitV0SignatureHash, CScript, OP_1, OP_CHECKMULTISIG, OP_0,OP_CHECKSIG
import sys
sys.path.append("..")
from ecc import PrivateKey

prev_tx_hx = int("8d98c41dcca391c915ee9ed8cc8d1b24da5abc76ab24c95a6f13b010cdef2228", 16)
prv_amount=int(100*100000000)
prv_pub_key_hash = "7d7143a28a3b836e3043f3ba5dbcdc4ee29b7457"
prv_script=bytes.fromhex(f"76a914{prv_pub_key_hash}88ac")
prev_index = 0

p=ECKey()
p.set(0x00db5cdc1663eea25406126e642f72ef7c78f68d249589d2873c613e61929eea)

target_public_key_bytes=bytes.fromhex(wallet[12]["public key"])
target_amount = int(99.999*100000000)

tx_in =CTxIn(COutPoint(prev_tx_hx, prev_index))

target_public_key=ECPubKey().set(target_public_key_bytes)
mast_scripts = [ CScript([target_public_key.get_bytes(), OP_CHECKSIG]) ]

locking_script, tweak, control_map = taproot_construct(target_public_key, mast_scripts)

tx_outs = CTxOut(target_amount, locking_script)

tx = CTransaction()
tx.vin=[tx_in]
tx.vout=[tx_outs]
tx.calc_sha256()
tx_sig_hash = SegwitV0SignatureHash(prv_script, tx, 0, 0x01, prv_amount)
tx_input_0_wit = CTxInWitness([p.sign_ecdsa(tx_sig_hash) + (1).to_bytes(1, 'big'), p.get_pubkey().get_bytes(False)])
tx.wit.vtxinwit = [tx_input_0_wit]

print(tx.serialize_with_witness().hex())

# Use for Next Transaction
tx1_outs = tx.vout
t = tweak
cmp = control_map


01000000000101b87af619fc3868147af6aaf45e1e5d41601eb280b326b5fbb129a7428b46ad710000000000ffffffff01605d0a5402000000225120d53a7554fa8af2453ed8ca4c88c227f55019b771f103c2bf9d2c2e9b341a81ce024730440220027b93455d1d980f7e67bee48ba97e4e05aaa5e03dfec77568c088f7f90886f902207afb8c5e9f00286d7f7b614f64a451311ec2e3ed5373758cc35c65453eee20a1012103eb477934368b96afb4e7878b96f29bc4aec8238189652f17a7b33eb0ed6acbee00000000


In [16]:
# last transaction to go on chain
from messages import ser_string
from script import get_p2wpkh_script, TaprootSignatureHash
from segwit_addr import decode_segwit_address
from key import SECP256K1_ORDER, SECP256K1, SECP256K1_G, int_or_bytes

taproot_locking_tx_id = int("f868c39f9f7e7c7c71f3c482815e9b09d317cc96bd8eb77c5bea24a1cca38d35", 16)
taproot_locking_tx_index = 0

taproot_unlocking_sec_key = ECKey().set(bytes.fromhex("13e4f9204068a232f23fe6305bb6c1fcb6d2c6e53347956a7fbaf2953737ab4c"))

target_amount = int(99.998*100000000)
target_b32_address1 = wallet[0]["address"]
target_b32_address2 = wallet[1]["address"]

tx_in_0 = CTxIn(COutPoint(taproot_locking_tx_id, taproot_locking_tx_index))
tx_ins = [tx_in_0]

locking_public_key_hash1 = bytes(decode_segwit_address("bcrt", target_b32_address1)[1])
locking_script1 = get_p2wpkh_script(locking_public_key_hash1)


locking_public_key_hash2 = bytes(decode_segwit_address("bcrt", target_b32_address2)[1])
locking_script2 = get_p2wpkh_script(locking_public_key_hash2)

tx_out_0 = CTxOut(target_amount//2, locking_script1)
tx_out_1 =CTxOut(target_amount//2,locking_script2)
tx_outs = [tx_out_0,tx_out_1]

tx = CTransaction()
tx.vin = tx_ins
tx.vout = tx_outs
tx.calc_sha256()

prev_tx_outs = tx1_outs
tx_sig_hash_0 = TaprootSignatureHash(tx, prev_tx_outs, 0X00, 0, True, list(cmp)[0])
taproot_unlocking_sec_key.negate()
tx_sign_0 = taproot_unlocking_sec_key.sign_schnorr(tx_sig_hash_0)

tx_input_0_wit = CTxInWitness([ tx_sign_0, list(cmp)[0], cmp[list(cmp)[0]]])
tx.wit.vtxinwit = [tx_input_0_wit]
print(tx.serialize_with_witness().hex())



01000000000101358da3cca124ea5b7cb78ebd96cc17d3099b5e8182c4f3717c7c7e9f9fc368f80000000000ffffffff02606b042a010000001600147d7143a28a3b836e3043f3ba5dbcdc4ee29b7457606b042a010000001600141701aa6015f935e1a695e902ace453af3ce4549703409653d3a17ec7a4ed1a5e368f278106af0a7e37f17f3869c9c065a2c328a0c6d52386cb94e370b04d9ce44b1faf6707621796da7acacee31544a409f04da4b2272220728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a19ac21c0728f0593227b1ba7d31914bf2f83d7335e12fdb92c7b39d12999594b61997a1900000000
